# GDP


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Read Data

In [7]:
economist = pd.read_hdf("data/democracy.h5", "economist")
freedom_house = pd.read_hdf("data/democracy.h5", "freedomhouse")
v_dem = pd.read_hdf("data/democracy.h5", "vdem")

score
country_name year       
Afghanistan  2006   3.06
             2008   3.02
             2010   2.48
             2011   2.48
             2012   2.48
...                  ...
eSwatini     2015   3.09
             2016   3.03
             2017   3.03
             2018   3.03
             2019   3.14

[2004 rows x 1 columns]

## Get GDP data
Note: All data is in PPP (as this is a much better measurement of wealth). Source: World Bank.

In [18]:
gdp = pd.read_csv("data/gdp.csv", skiprows=[0, 1, 2, 3], index_col=[0])
gdp.drop(["Country Code", "Indicator Name", "Indicator Code", "Unnamed: 65"], axis=1, inplace=True)
gdp.index.set_names("country_name", inplace=True)
gdp

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
country_name,,,,,,,,,,,,,,,,,,,,,
Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.502846e+09,3.442856e+09,3.799467e+09,3.816822e+09,3.893071e+09,3.941354e+09,4.098240e+09,NaN,NaN,NaN
Africa Eastern and Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.769027e+12,1.771196e+12,1.891955e+12,2.022763e+12,2.094684e+12,2.208686e+12,2.316062e+12,2.434969e+12,2.532315e+12,2.490299e+12
Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.118418e+10,5.966700e+10,6.503984e+10,6.905834e+10,7.183170e+10,7.009796e+10,7.471192e+10,7.741557e+10,8.187980e+10,8.088821e+10
Africa Western and Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.324190e+12,1.396677e+12,1.526772e+12,1.645122e+12,1.662297e+12,1.678674e+12,1.744087e+12,1.841811e+12,1.937451e+12,1.945572e+12
Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.625388e+11,1.861242e+11,1.998656e+11,2.203648e+11,2.046036e+11,2.048747e+11,2.179873e+11,2.187486e+11,2.212628e+11,2.117584e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.393504e+10,1.452770e+10,1.524200e+10,1.567947e+10,1.673239e+10,1.778924e+10,1.869121e+10,1.979201e+10,2.110361e+10,2.020985e+10
"Yemen, Rep.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.587612e+10,8.544086e+10,9.275561e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
South Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.965262e+11,6.982210e+11,7.305176e+11,7.419156e+11,7.589010e+11,7.727687e+11,7.901706e+11,8.211793e+11,8.367836e+11,7.921031e+11
